In [1]:
%env CUDA_VISIBLE_DEVICES=7

import os
import logging

os.environ.pop("HF_HUB_OFFLINE", None)
logging.getLogger().setLevel(logging.ERROR)  # or logging.CRITICAL

import torch
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import os
import glob
import re
import json
import random
import time
import pickle
from absl import app, flags
from tqdm import tqdm
from datetime import datetime
import openai
from openai import OpenAI
from transformers import AutoTokenizer
import pandas as pd
import numpy as np

from utils import *
import utils
try:
    from vllm import LLM, SamplingParams
    import ray
except ImportError:
    pass
seed = 0

env: CUDA_VISIBLE_DEVICES=7
INFO 04-26 18:43:57 __init__.py:190] Automatically detected platform cuda.


In [2]:
import subprocess
import torch
def get_freest_cuda_device():
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=memory.free', '--format=csv,nounits,noheader'],
        stdout=subprocess.PIPE, encoding='utf-8')
    memory_free = [int(x) for x in result.stdout.strip().split('\n')]
    return memory_free.index(max(memory_free))

best_gpu = get_freest_cuda_device()
device = torch.device(f"cuda:{best_gpu}")
print(f"Using GPU: {device}")
# %env CUDA_VISIBLE_DEVICES=0

Using GPU: cuda:3


In [2]:
with open(os.path.abspath('../ryan_openai.txt'), 'r') as f:
    utils.client = OpenAI(api_key=f.read().rstrip('\n'))

In [3]:
import os

# Explicitly unset all offline-related env vars
os.environ.pop("HF_HUB_OFFLINE", None)
os.environ.pop("TRANSFORMERS_OFFLINE", None)
os.environ["HF_HUB_OFFLINE"] = "0"
os.environ["TRANSFORMERS_OFFLINE"] = "0"

with open("../token.txt", "r") as f:
    token = f.read().strip()

from huggingface_hub import login
login(token=token)

FileNotFoundError: [Errno 2] No such file or directory: '../token.txt'

In [4]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("Tianyi-Lab/Personas")
personas_chatting_array = ds['train']['Llama-3.1-70B-Instruct_descriptive_persona']

In [5]:
np.random.seed(seed)
random_indices = np.random.choice(len(personas_chatting_array), size=200, replace=False).astype(int)
personas_chatting_array = np.array(personas_chatting_array)[random_indices]

In [6]:
import json
import shutil
import os
config_chatting = {'agent1_role': 'Person A',
                  'agent2_role': 'Person B',
                  'agent1_prompt': "You are %SPEAKER_ROLE%, and you are having an online conversation with %LISTENER_ROLE%. Each of you is chatting to get to know each other, taking turns asking questions, and sharing stories about your lives, careers, and experiences. The goal is to try to find something in common. Do not only ask questions, you should also share details about yourself. This is a brief story that you have written about yourself: %SPEAKER_BACKSTORY%",
                  'agent2_prompt': "You are %SPEAKER_ROLE%, and you are having an online conversation with %LISTENER_ROLE%. Each of you is chatting to get to know each other, taking turns asking questions, and sharing stories about your lives, careers, and experiences. The goal is to try to find something in common. Do not only ask questions, you should also share details about yourself. This is a brief story that you have written about yourself: %SPEAKER_BACKSTORY%",
                  'reminder_prompt': "Keep your response very brief — 2 sentences or less. Do NOT repeat anything you've already said: \n",
                  "scenario": "Two people are getting to know one another through an online conversation. ",
                  'eval_prompts': {
                  'prompt_consistency': """You are evaluating whether the following utterance by %SPEAKER_ROLE% is consistent with their given background and the task instructions. Answer YES if the line directly contradicts any part of the background or task descriptio—this includes contradictions in facts, behavior, beliefs, emotional stance, or conversational strategy. Answer NO if the line introduces new details, but those details are **plausibly aligned** with the background. Be strict and literal in your evaluation: if the line violates the persona’s stated traits, mark it as a contradiction even if the violation is subtle. Background for %SPEAKER_ROLE%: %SPEAKER_BACKSTORY% Line spoken by %SPEAKER_ROLE%: %SPEAKER_LINE%  Answer YES if the line contradicts the background, and NO if it does not. Then, explain your answer in one sentence. Be precise and avoid vague justification."""}}

os.makedirs("chatting", exist_ok=True)
with open("chatting/config_chatting.json", "w", encoding="utf-8") as f:
    json.dump(config_chatting, f, indent=4)

In [7]:
llms = ["Llama-3.1-8B-Instruct", "gpt-4o-mini", "Qwen2.5-3B-Instruct", "Llama-3.1-8B", "Mistral-7B-Instruct", "Llama-3.1-70B", "Llama-3.1-70B-Instruct", "phi-3.5-mini-instruct"]
        
config_llm = {'agent1_model': 'gemma-2-2b-it',
             'agent2_model': 'gemma-2-2b-it',
             'eval_model': 'Llama-3.1-8B-Instruct',
             'iterations': 10,
             'verbose': False,
             'write': True,
             'convo_length_limit': 10,
             'max_tokens': 256,
             'gpus': 1,
             'seed': 0,
             'task_name': 'Chatting',
             'model_dir': "/raid/users/ryan_cheng/models/"}

# with open("chatting/Llama-3.1-8B-Instruct.json", "w", encoding="utf-8") as f:
#     json.dump(config_llm, f, indent=4)

In [8]:
# def get_text_after_colon(label, text):
#     """
#     Extracts the text that appears after a given label followed by a colon.
#     Example: get_text_after_colon("Name", "Name: John Doe") returns "John Doe"
#     """
#     pattern = rf"{re.escape(label)}:\s*(.*)"
#     match = re.search(pattern, text)
#     return match.group(1).strip() if match else text

# personas_chatting = []

# for persona in personas_chatting_array:
#     name_prompt = "Given the following biography below, parse the name of the person. \nOnly output the name.\n" + persona + "\nName: "
#     name = completion_create("gpt-4o-mini", config_llm, name_prompt)
#     name = get_text_after_colon("Name", name)
#     personas_chatting.append({'persona': persona, 'name': name})

In [9]:
# with open('chatting/config_chatting_personas.json', 'w') as f:
#     json.dump(personas_chatting, f, indent=4)

In [8]:
with open("chatting/config_chatting_personas.json", "r") as f:
    personas_chatting = json.load(f)

In [ ]:
import re

def get_first_name(full_name):
    return full_name.strip().split()[0]
    
def clean_role_prefix(response, expected_role):
    """
    Removes repeated instances of the expected_role prefix at the start (e.g., 'Therapist: Therapist:'),
    and ensures the response begins with a single correct expected_role prefix.
    """
    pattern = rf"^(({re.escape(expected_role)}):\s*)+"
    cleaned = re.sub(pattern, '', response.strip(), flags=re.IGNORECASE)
    return cleaned
    
def is_role_confused(response, other_role):
    """
    Checks if the output starts with the wrong speaker tag.
    """
    if other_role + ":" in response:
        return True
    else: 
        return False

def generate_response(agent_model, expected_role, other_role, config_llm, prompt, max_retries=3):
    for _ in range(max_retries):
        response = completion_create(agent_model, config_llm, prompt)
        print(expected_role)
        if not is_role_confused(response, other_role):
            return clean_role_prefix(response, expected_role)
            
    return clean_role_prefix(response, expected_role)

def generate_chat(config_llm, p1, p2, p1_name, p2_name, pturn=1):
    stats['P1'] = p1
    stats['P2'] = p2
    stats['pturn'] = pturn
    config_chatting["agent1_role"] = get_first_name(p1_name)
    config_chatting["agent2_role"] = get_first_name(p2_name)

    round_num = 0
    while round_num < config_llm['convo_length_limit']:
        conversation = ("".join([turn[1] if isinstance(turn, tuple) else turn for turn in stats["conversation"]]) if len(stats["conversation"]) != 0 else "You are starting the conversation.\n")
        
        if pturn == 1:
            prompt = config_chatting["agent1_prompt"]
            pturn = 2
            if config_llm["verbose"]:
                print(prompt)
                print()

            if round_num!=0: 
                prompt+= "Your conversation so far is below:\nConversation: %CONVERSATION%"
                
            if round_num >=config_llm['convo_length_limit']*2-11 and round_num<=config_llm['convo_length_limit']*2-1:
                prompt+= "You have " + str((config_llm['convo_length_limit']-round_num)//2) + " rounds left." + "Make sure to conclude the conversation as you're near the end."

            elif round_num>config_llm['convo_length_limit']*2-1:
                prompt+= "This is your concluding line in the conversation."

            if round_num!=0: 
                prompt+= "Continue the conversation with " + config_chatting["agent2_role"] +  ". Remember you are " +  config_chatting["agent1_role"] + "."
                 
            prompt += config_chatting["reminder_prompt"] + "DO NOT PREFACE THE RESPONSE WITH THIRD-PERSON STATEMENTS SUCH AS \"Sure, here's a response from...\"\n"
            prompt+="%SPEAKER_ROLE%:"
            prompt = prompt.replace("%SPEAKER_ROLE%", config_chatting["agent1_role"]) \
                           .replace("%LISTENER_ROLE%", config_chatting["agent2_role"]) \
                           .replace("%SPEAKER_BACKSTORY%", p1) \
                           .replace("%CONVERSATION%", conversation)
            
            response = generate_response(config_llm['agent1_model'], config_chatting["agent1_role"], config_chatting["agent2_role"], config_llm, prompt)
            while "\n\n" in response:
                print(response)
                response = generate_response(config_llm['agent1_model'], config_chatting["agent1_role"], config_chatting["agent2_role"], config_llm, prompt)
            stats["conversation"].append((round_num, f"{config_chatting["agent1_role"]}: " + response + "\n"))
        
        else:
            prompt = config_chatting["agent2_prompt"]
            pturn = 1    
            if config_llm["verbose"]:
                print(prompt)
                print()

            if round_num!=0: 
                prompt+= "Your conversation so far is below:\nConversation: %CONVERSATION%"
            if round_num >=config_llm['convo_length_limit']*2-11 and round_num<=config_llm['convo_length_limit']*2-1:
                prompt+= "You have " + str((config_llm['convo_length_limit']-round_num)//2) + " rounds left." + "Make sure to conclude the conversation as you're near the end."
            elif round_num>config_llm['convo_length_limit']*2-1:
                prompt+= "This is your concluding line in the conversation."

            if round_num!=0: 
                prompt+= "Continue the conversation with " + config_chatting["agent1_role"] +  ". Remember you are " +  config_chatting["agent2_role"] + "."

            prompt += config_chatting["reminder_prompt"] + "DO NOT PREFACE THE RESPONSE WITH THIRD-PERSON STATEMENTS SUCH AS \"Sure, here's a response from...\"\n"
            prompt+="%SPEAKER_ROLE%:"
            prompt = prompt.replace("%SPEAKER_ROLE%", config_chatting["agent2_role"]) \
                           .replace("%LISTENER_ROLE%", config_chatting["agent1_role"]) \
                           .replace("%SPEAKER_BACKSTORY", p2) \
                           .replace("%CONVERSATION%", conversation)

            response = generate_response(config_llm['agent1_model'], config_chatting["agent1_role"], config_chatting["agent2_role"], config_llm, prompt)
            
            while "\n\n" in response:
                print(response)
                response = generate_response(config_llm['agent2_model'], config_chatting["agent2_role"], config_chatting["agent1_role"], config_llm, prompt)
            stats["conversation"].append((round_num, f"{config_chatting["agent2_role"]}: " + response + "\n"))
        round_num += 1

    stats["rounds"] = round_num
    if config_llm['verbose']:
        print(stats["conversation"])
    return stats.copy()

def reset_stats():
    stats_template = {
        "task_name": config_llm['task_name'],
        "P1": "",
        "P2": "",
        "conversation": [],
        "pturn": 0, # beginning person (1 or 2)
        "index": -1,
        "timestamp": "",
        "rounds": 0,
        'conversation_only': True
    }
    for key, value in stats_template.items():
        stats[key] = value

In [10]:
import os
import random
from datetime import datetime
import utils
utils.config = config_llm

current_date = str(datetime.now().strftime("%m.%d.%y"))
output_dir = f"chatting/exp/{current_date}"
os.makedirs(output_dir, exist_ok=True)

# Generate unique random number for filename
def generate_unique_file_number(output_dir, prefix, seed, extension=".json"):
    while True:
        rand_num = random.randint(0, 1000)
        filename = f"{prefix}_{seed}_{rand_num}{extension}"
        filepath = os.path.join(output_dir, filename)
        if not os.path.exists(filepath):
            return rand_num

unique_num = generate_unique_file_number(
    output_dir,
    config_llm['agent1_model'],
    config_llm['seed']
)

# File to write output to
write_file = os.path.join(output_dir, f"{config_llm['agent1_model']}_{config_llm['seed']}_{unique_num}.json")

In [11]:
lengths = [10, 20, 40, 60]
write_files = [os.path.join(output_dir, f"{config_llm['agent1_model']}_{config_llm['seed']}_{length}_{unique_num}.json") for length in lengths]

In [12]:
len(personas_chatting)

200

In [19]:
# index_offset = load_stats_file(write_file)
conversations = []    

for i in range(1):
    for p_dict1, p_dict2 in tqdm(np.array(personas_chatting).reshape(-1, 2)):
        for j, convo_length in enumerate(lengths):
            index_offset = load_stats_file(write_files[j])
            config_llm['convo_length_limit'] = convo_length
            reset_stats()
            conversation = generate_chat(
                config_llm,
                p_dict1["persona"], 
                p_dict2["persona"],
                p_dict1["name"], 
                p_dict2["name"], 
                pturn=1
            )
            print(conversation)
            conversations.append(conversation)
            stats['index'] = index_offset
            stats['timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            write_stats(write_files[j])
            index_offset += 1


  0%|          | 0/100 [00:00<?, ?it/s]



Processed prompts: 100%|██████████| 2/2 [00:00<00:00,  6.17it/s, est. speed input: 4678.77 toks/s, output: 388.82 toks/s]


Alexandra
Sure, here's a brief summary about me:

I am a passionate and driven individual who grew up in Charleston, South Carolina, and now runs my own sustainable landscaping business. I am a dedicated mother to my 12-year-old son and find solace in exploring the beauty of nature and contributing to my community.






Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.97it/s, est. speed input: 3025.06 toks/s, output: 239.44 toks/s]


Alexandra
Sure, here's a brief overview of myself:

I'm a passionate advocate for the environment and sustainability, and I'm deeply connected to my roots in the Lowcountry. I'm a strong and determined individual who thrives on challenges and enjoys connecting with others through community service.






Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.55it/s, est. speed input: 2699.17 toks/s, output: 252.81 toks/s]


...

Ethan






Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  7.61it/s, est. speed input: 5637.76 toks/s, output: 252.76 toks/s]


Ethan




  1%|          | 1/100 [01:37<2:41:04, 97.62s/it]


KeyboardInterrupt: 

In [ ]:
def eval_prompt_consistency(conv_dict):
    #assert 'eval_prompt_consistency' not in conv_dict # warn if we are replacing metrics we don't mean to overwrite
    conv_dict['eval_prompt_consistency'] = []
    conv_dict['P1_prompt_consistency_score'] = 0
    conv_dict['P2_prompt_consistency_score'] = 0
    p2_utterances = 0
    p1_utterances = 0
    pturn = conv_dict["pturn"]
    for line in conv_dict["conversation"]:
        line_number = line[0]
        convo_line = line[1]
        if pturn == 1:
            prompt = config_chatting["eval_prompts"]["prompt_consistency"].replace("%SPEAKER_ROLE%", config_chatting["agent1_role"]) \
                                                                          .replace("%SPEAKER_BACKSTORY%", conv_dict["P1"]) \
                                                                          .replace("%SPEAKER_LINE%", convo_line)
            if config_llm['verbose']:
                print(prompt)
            output = completion_create(config_llm['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append((line_number, output))
            if "YES" not in output: # no contradiction
                conv_dict['P1_prompt_consistency_score'] += 1
            p1_utterances += 1
            pturn = 2
        elif pturn == 2:
            prompt = config_chatting["eval_prompts"]["prompt_consistency"].replace("%SPEAKER_ROLE%", config_chatting["agent2_role"]) \
                                                                          .replace("%SPEAKER_BACKSTORY%", conv_dict["P2"]) \
                                                                          .replace("%SPEAKER_LINE%", convo_line)
            if config_llm['verbose']:
                print(prompt)
            output = completion_create(config_llm['eval_model'], config, prompt)
            conv_dict['eval_prompt_consistency'].append((line_number, output))
            if "YES" not in output: # no contradiction
                conv_dict['P2_prompt_consistency_score'] += 1
            p2_utterances += 1
            pturn = 1 
            
    if p1_utterances > 0:
        conv_dict['P1_prompt_consistency_score'] /= p1_utterances

    if p2_utterances > 0:
        conv_dict['P2_prompt_consistency_score'] /= p2_utterances
        
    print(conv_dict)
    return conv_dict

In [ ]:
import utils
utils.config = config_llm

with open("chatting/exp/04.22.25/Llama-3.1-8B-Instruct_0_724.json", "r") as f:
    conversations = json.load(f)

for conversation in tqdm(conversations):
    conversation = eval_prompt_consistency(conversation)

write_stats(write_file)

In [ ]:
with open(write_file, "w", encoding="utf-8") as f:
    json.dump(conversations, f, indent=4)

In [ ]:
conversations